In [1]:
import json
import hashlib
from datetime import datetime

# -----------------------------
# Encryption / Decryption helpers
# -----------------------------
def encrypt(message, key):
    """Simple XOR encryption for simulation (not real crypto)."""
    return ''.join(chr(ord(c) ^ ord(key[i % len(key)])) for i, c in enumerate(message))

def decrypt(ciphertext, key):
    """Symmetric XOR decryption (same as encrypt)."""
    return encrypt(ciphertext, key)

# -----------------------------
# Key Distribution Center (KDC)
# -----------------------------
class KDC:
    def __init__(self):
        # Shared secret keys with client and servers
        self.keys = {
            "alice": "key_alice",
            "tgs": "key_tgs",
            "fileserver": "key_fs"
        }

    def authenticate_client(self, client_id):
        """AS: Step 1 & 2 — create TGT and session key for C–TGS"""
        k_c_tgs = "session_C_TGS"
        ticket_tgs = {
            "client": client_id,
            "session_key": k_c_tgs,
            "timestamp": str(datetime.now())
        }
        enc_tgt = encrypt(json.dumps(ticket_tgs), self.keys["tgs"])
        return enc_tgt, encrypt(k_c_tgs, self.keys[client_id])

    def issue_service_ticket(self, tgt_enc, authenticator_enc, service_id):
        """TGS: Step 3 & 4 — validate TGT and issue Service Ticket"""
        tgt = json.loads(decrypt(tgt_enc, self.keys["tgs"]))
        k_c_tgs = tgt["session_key"]

        auth = json.loads(decrypt(authenticator_enc, k_c_tgs))
        client_id = auth["client"]

        # Create session key between client and service
        k_c_v = "session_C_FS"
        service_ticket = {
            "client": client_id,
            "session_key": k_c_v,
            "timestamp": str(datetime.now())
        }
        enc_service_ticket = encrypt(json.dumps(service_ticket), self.keys[service_id])
        return enc_service_ticket, encrypt(k_c_v, k_c_tgs)

# -----------------------------
# Service Server (SS)
# -----------------------------
class ServiceServer:
    def __init__(self, server_id, key):
        self.server_id = server_id
        self.key = key

    def verify_ticket(self, ticket_enc, authenticator_enc):
        """Step 5 — Verify the ticket and client."""
        ticket = json.loads(decrypt(ticket_enc, self.key))
        k_c_v = ticket["session_key"]

        auth = json.loads(decrypt(authenticator_enc, k_c_v))
        client_id = auth["client"]

        print(f"[Service] Authenticated client {client_id}")
        reply = {"ok": True, "timestamp": str(datetime.now())}
        return encrypt(json.dumps(reply), k_c_v)

# -----------------------------
# Client
# -----------------------------
class Client:
    def __init__(self, client_id, key):
        self.client_id = client_id
        self.key = key
        self.k_c_tgs = None
        self.k_c_v = None

    def request_tgt(self, kdc):
        print("[Client] Requesting TGT from AS...")
        tgt_enc, k_c_tgs_enc = kdc.authenticate_client(self.client_id)
        self.k_c_tgs = decrypt(k_c_tgs_enc, self.key)
        print("[Client] Obtained TGT and session key (C–TGS).")
        return tgt_enc

    def request_service_ticket(self, kdc, tgt_enc, service_id):
        print(f"[Client] Requesting service ticket for {service_id}...")
        auth = json.dumps({"client": self.client_id, "timestamp": str(datetime.now())})
        auth_enc = encrypt(auth, self.k_c_tgs)
        enc_st, k_c_v_enc = kdc.issue_service_ticket(tgt_enc, auth_enc, service_id)
        self.k_c_v = decrypt(k_c_v_enc, self.k_c_tgs)
        print("[Client] Obtained service ticket and session key (C–S).")
        return enc_st

    def access_service(self, service, service_ticket_enc):
        print("[Client] Accessing service...")
        auth = json.dumps({"client": self.client_id, "timestamp": str(datetime.now())})
        auth_enc = encrypt(auth, self.k_c_v)
        reply_enc = service.verify_ticket(service_ticket_enc, auth_enc)
        reply_plain = decrypt(reply_enc, self.k_c_v)
        print("[Client] Service reply:", reply_plain)

# -----------------------------
# Simulation
# -----------------------------
kdc = KDC()
client = Client("alice", "key_alice")
service = ServiceServer("fileserver", "key_fs")

# 1) Client requests TGT
tgt_enc = client.request_tgt(kdc)

# 2) Client requests service ticket
st_enc = client.request_service_ticket(kdc, tgt_enc, "fileserver")

# 3) Client uses service ticket to access service
client.access_service(service, st_enc)


[Client] Requesting TGT from AS...
[Client] Obtained TGT and session key (C–TGS).
[Client] Requesting service ticket for fileserver...
[Client] Obtained service ticket and session key (C–S).
[Client] Accessing service...
[Service] Authenticated client alice
[Client] Service reply: {"ok": true, "timestamp": "2025-11-04 09:30:02.905901"}


In [7]:
# ============================================
# Interactive Kerberos Simulation (Jupyter Safe)
# ============================================

import time
import secrets
from IPython.display import clear_output  # helps avoid output flooding in Jupyter

# ---------- User and Service Databases ----------
users = {
    "alice": "password123",
    "bob": "mypassword",
    "charlie": "abc123"
}

services = ["file_server", "email_server", "database_server"]

# ---------- Helper ----------
def generate_key():
    """Generate a random 8-byte session key (hex)."""
    return secrets.token_hex(8)


# ---------- Servers ----------
def authentication_server(username, password):
    print("\n[Authentication Server]")
    if username in users and users[username] == password:
        tgt = {
            "username": username,
            "issue_time": time.time(),
            "expiry_time": time.time() + 60,  # valid for 60 seconds
            "session_key": generate_key()
        }
        print(f"✅ Authentication successful for {username}")
        print("🎫 Ticket Granting Ticket (TGT) issued — valid for 60 seconds.")
        return tgt
    else:
        print("❌ Invalid username or password.")
        return None


def ticket_granting_server(tgt, service):
    print("\n[Ticket Granting Server]")
    if not tgt:
        print("❌ No valid TGT. Please authenticate first.")
        return None

    if time.time() > tgt["expiry_time"]:
        print("⚠️ TGT expired. Please reauthenticate.")
        return None

    if service not in services:
        print("❌ Invalid service request.")
        return None

    service_ticket = {
        "username": tgt["username"],
        "service": service,
        "issue_time": time.time(),
        "expiry_time": time.time() + 45,  # valid for 45 seconds
        "service_session_key": generate_key()
    }
    print(f"🎟 Service Ticket issued for '{service}' — valid for 45 seconds.")
    return service_ticket


def service_server(ticket):
    print(f"\n[{ticket['service'].upper()} SERVICE]")
    if time.time() > ticket["expiry_time"]:
        print("❌ Ticket expired. Access denied.")
    else:
        print(f"✅ Access granted to {ticket['username']}")
        print(f"🔑 Service Session Key: {ticket['service_session_key']}")
        print(f"📂 Welcome to {ticket['service']}!")


# ---------- Main Interactive Simulation ----------
def kerberos_simulation():
    tgt = None
    service_ticket = None
    username = None

    while True:
        clear_output(wait=True)
        print("========== KERBEROS MENU ==========")
        print("1. Login (Get TGT)")
        print("2. Request Service Ticket")
        print("3. Access Service")
        print("4. Check Ticket Expiry")
        print("5. Exit")
        print("===================================")

        choice = input("Enter your choice (1-5): ").strip()

        clear_output(wait=True)

        if choice == '1':
            username = input("Enter username: ").strip()
            password = input("Enter password: ").strip()
            tgt = authentication_server(username, password)
            input("\nPress Enter to continue...")

        elif choice == '2':
            if not tgt:
                print("⚠️ Please login first to obtain a TGT.")
                input("\nPress Enter to continue...")
                continue
            print("\nAvailable Services:")
            for s in services:
                print(" -", s)
            service = input("Enter the service you want to access: ").strip()
            service_ticket = ticket_granting_server(tgt, service)
            input("\nPress Enter to continue...")

        elif choice == '3':
            if not service_ticket:
                print("⚠️ Please request a service ticket first.")
                input("\nPress Enter to continue...")
                continue
            service_server(service_ticket)
            input("\nPress Enter to continue...")

        elif choice == '4':
            if tgt:
                remaining_tgt = int(tgt["expiry_time"] - time.time())
                print(f"⏳ TGT valid for: {max(0, remaining_tgt)} seconds")
            else:
                print("⚠️ No active TGT found.")

            if service_ticket:
                remaining_st = int(service_ticket["expiry_time"] - time.time())
                print(f"⏳ Service Ticket valid for: {max(0, remaining_st)} seconds")
            elif not tgt and not service_ticket:
                print("No tickets to check.")
            input("\nPress Enter to continue...")

        elif choice == '5':
            print("👋 Exiting Kerberos simulation. Goodbye!")
            break

        else:
            print("❌ Invalid choice. Please enter a valid option.")
            input("\nPress Enter to continue...")


# ---------- Run Simulation ----------
kerberos_simulation()


👋 Exiting Kerberos simulation. Goodbye!
